In [1]:
from infrastructure.models import (
    CuratedProjectCollection,
    Project,
    ProjectStatus,
    Initiative,
    InfrastructureType,
    PowerPlant,
)
from locations.models import (
    LineStringGeometry,
    PointGeometry,
    PolygonGeometry,
    GeometryStore,
    Region,
    Country,
)
from django.db.models import (
    Case, CharField, Count, ExpressionWrapper, F, FloatField, Q, Value, When
)
from django.contrib.postgres.aggregates import StringAgg
from django.db.models.functions import Cast
from decimal import Decimal
from django.db.models.functions import Lower

In [3]:
# Prod DB doesn't contain any power plants related to projects that have initiatives
pp_with_initiatives = PowerPlant.objects.annotate(
    initiative_count=Count('project__initiatives')
).filter(
    initiative_count__gt=0
).order_by('-initiative_count')
list(pp_with_initiatives.values_list('name', 'project__name', 'initiative_count')[:15])

[]

In [4]:
# For testing purposes, let's make a relationship
# First get a list of all projects with initiatives
projects_with_initiatives = Project.objects.annotate(
    initiative_count=Count('initiatives')
).filter(
    initiative_count__gt=0
).order_by('-initiative_count')
list(projects_with_initiatives.values_list('name', 'initiative_count')[:15])

[('Sofia Northern Speed Tangent', 5),
 ('Corridor VC Mostar South', 4),
 ('M5 Upgrade (Bobruysk - Zhlobin)', 4),
 ('Xinjiang Regional Road Improvement Project', 4),
 ('Dali-Ruili Railway', 4),
 ('Lot 1 Svilaj on The Sava River (connection to Corridor X) - Doboj South',
  3),
 ('Vidin Calafat Danube Bridge (Road) (Bulgaria)', 3),
 ('Raikot - Khunjerab Karakoram Highway (Upgrade)', 3),
 ('Mae Sot/Myawady-Pa an-Thaton', 3),
 ('Thakot - Raikot Karakoram Highway (Upgrade)', 3),
 ('Lot 2: Doboj (South) - Sarajevo South (Tarčin)', 3),
 ('Yuxi-Mohan Subregional Railway Link Project', 3),
 ('Sainshand - Ereentsav, Nomrog, Bichil Rail Route', 3),
 ('Gwadar Port Construction of Breakwaters', 3),
 ('Gwadar Port - Infrastructure for Free Zone & EPZs port-related industries',
  3)]

In [5]:
# Now grab a random power plant
kursk = PowerPlant.objects.get(name='Kursk Nuclear Power Plant')
kursk

<PowerPlant: Kursk Nuclear Power Plant>

In [6]:
# There are currently no initatives related to this power plant's projects
Initiative.objects.filter(project__power_plant_id=kursk.pk)

<TreeQuerySet []>

In [10]:
# link a project from above to Kursk
dali_railway = Project.objects.get(name='Dali-Ruili Railway')
dali_railway.power_plant = kursk
dali_railway.save()
dali_railway

<Project: Dali-Ruili Railway>

In [12]:
# Now we can re-run the count from above and see that it returns a power plant now
pp_with_initiatives = PowerPlant.objects.annotate(
    initiative_count=Count('project__initiatives')
).filter(
    initiative_count__gt=0
).order_by('-initiative_count')
list(pp_with_initiatives.values_list('name', 'project__name', 'initiative_count')[:15])

[('Kursk Nuclear Power Plant', 'Dali-Ruili Railway', 4)]

In [13]:
# now we can get the related initiatives to the Kursk power plant
initiatives = Initiative.objects.filter(project__power_plant_id=kursk.pk)
initiatives

<TreeQuerySet [<Initiative: Belt and Road>, <Initiative: GMS North-South Economic Corridor>, <Initiative: Trans-Asian Railway>, <Initiative: Singapore-Kunming Rail Link>]>